# Applying Google AI on Classifying Data from Job Post

## Case scenerio:
There are numerous course alternatives available to students within the same major before they enroll in a university, academy, or college. There is a lack of specialized knowledge and skills in real-life jobs, however, resulting from the majority of these courses' shallow instruction, which makes it challenging for students to obtain them. On the other hand, a lack of highly qualified workers for many organizations in emerging nations makes hiring difficult, particularly for senior roles that still need more talent. By using Google Gen AI to categorize data and skill sets, this initiative hopes to assist educational institutions in identifying and improving the specialized skills and tools required in their curricula so that students are better equipped for the workforce.

📄 **Use case of project**:
- Using Selenium with Beutiful Soup to collect data from career website
- Classifying different categories from job post using Google Generative AI

🤖 **Gen AI capabilities**:
- Document understanding: Understanding different job post
- Structured output/JSON mode: Return result as JSON
- Few-shot prompting: Classifying data

In [ ]:
# Install required packages
# %pip install pandas selenium tldextract beutyfulsoup4 requests
# %pip install -U -q "google-genai==1.7.0"

Callout libraries

In [1]:
import pandas as pd
import re
from google import genai

import time
from datetime import datetime, timedelta

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
)

import tldextract
import httpx
from bs4 import BeautifulSoup

from configparser import ConfigParser

import json

from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

Read config file

In [33]:
config = ConfigParser()
config.read('ConfigFile.properties')

['ConfigFile.properties']

Setup variables

In [34]:
today =  datetime.today().strftime('%Y-%m-%d')
company_url = config.get('URL', 'url.broadcom')
googleai_key = config.get('API', 'api.key_googleai')
company = "Broadcom"
webdriver_service = Service(config.get('URL', 'url.chromedriver')) 

Define functions:
- **collect_job_links()**: function to collect career links from Broadcom career website
- **convert_posted_date()**: the date on career website are being set as: *"Posted Today", "Posted Yesterday", "Posted 2 days ago", etc.*, so the function will converted to date as type ***%d-%m-&Y***

In [35]:
def collect_job_links(driver):
    job_urls = []
    while True:
        try:
            job_links = driver.find_elements(By.XPATH, "//a[@data-automation-id='jobTitle']")
            job_urls.extend([link.get_attribute("href") for link in job_links])
            
            next_button = driver.find_element(By.XPATH, "//button[@data-uxi-element-id='next']")
            
            if not next_button.is_enabled():
                break
            
            next_button.click()
            time.sleep(5)
            
        except NoSuchElementException:
            print("No more pages or next button not found, stopping...")
            break
        except TimeoutException:
            print("Timeout while loading page, stopping...")
            break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break
    
    return job_urls

In [36]:
def convert_posted_date(posted_str):
    today = datetime.today()
    
    if 'Today' in posted_str:
        return today.strftime("%d-%m-%Y")
    elif 'Yesterday' in posted_str:
        return (today - timedelta(days=1)).strftime("%d-%m-%Y")
    
    match = re.search(r'(\d+)', posted_str)
    if match:
        days_ago = int(match.group(1))
        return (today - timedelta(days=days_ago)).strftime("%d-%m-%Y")
    
    return None

In [ ]:
driver = webdriver.Chrome()
time.sleep(5)  
company = tldextract.extract(company_url).domain
driver.get(company_url)
driver.maximize_window()
time.sleep(5)
try:
    cookie_banner = driver.find_element(By.ID, "onetrust-policy")
    dismiss_button = cookie_banner.find_element(By.XPATH, "//button[contains(text(), 'Accept') or contains(text(), 'I Understand')]") # Change the text if needed
    dismiss_button.click()  # Accept cookies to remove the banner
except NoSuchElementException:
    print("No cookie banner found, proceeding...")
time.sleep(3)

try:
    job_links = collect_job_links(driver)
except NoSuchElementException:
    print("No job links found, proceeding...")
except Exception as e:
    print(e)

driver.quit()

KeyboardInterrupt: 

In [ ]:
job_links_df = pd.DataFrame(job_links, columns=["Job Links"])
# For backup purposes, save the job links to a CSV file
job_links_df.to_csv("Broadcom_job_links.csv", index=False, encoding='utf-8-sig')

In [ ]:
# In case that the ipynb restarts from the beginning, uncomment the following line to read the job links from the CSV file
# job_links_df = pd.read_csv("Broadcom_job_links.csv", encoding='utf-8-sig')

In [40]:
all_job_data = []
# For test case extract first 10 job links
job_links_test = job_links_df['Job Links'][:10]
driver3 = webdriver.Chrome(service=webdriver_service)
with httpx.Client(timeout=10) as client:
    for link in job_links_df['Job Links']:
        driver3.get(link)
        time.sleep(3)
        try:
            soup = BeautifulSoup(driver3.page_source, 'html.parser')
            time.sleep(2)
            job_metadata = {}
            
            job_title_tag = soup.find("h2", {"data-automation-id": "jobPostingHeader"})
            job_title = job_title_tag.get_text(strip=True) if job_title_tag else "N/A"

            job_location_div = soup.find("div", {"data-automation-id": "locations"})
            job_locations = []
            if job_location_div:
                locations = job_location_div.find_all("dd", class_="css-129m7dg")
                job_locations = [loc.get_text(strip=True) for loc in locations]
            job_location = "\n ".join(job_locations) if job_locations else "N/A"

            date_posted_div = soup.find("div", {"data-automation-id": "postedOn"})
            date_posted = "N/A"
            if date_posted_div:
                date_tag = date_posted_div.find("dd", class_="css-129m7dg")
                date_posted = date_tag.get_text(strip=True) if date_tag else "N/A"
            date_posted = convert_posted_date(date_posted)

            job_id_div = soup.find("div", {"data-automation-id": "requisitionId"})
            job_id = "N/A"
            if job_id_div:
                job_id_tag = job_id_div.find("dd", class_="css-129m7dg")
                job_id = job_id_tag.get_text(strip=True) if job_id_tag else "N/A"

        
            job_type_div = soup.find("div", {"data-automation-id": "time"})
            job_type = job_type_div.get_text(strip=True) if job_type_div else "N/A"
            if job_type != "N/A":
                job_type = job_type[9::]

            job_description_elem = WebDriverWait(driver3, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-automation-id='jobPostingDescription']"))
            )
            job_text = job_description_elem.find_elements(By.CSS_SELECTOR, "p, ul, ul li")
            job_description = "\n".join([p.text for p in job_text])

            job_data = {
                "Company": company,
                "Job Title": job_title,
                "Job Location": job_location,
                "Date Posted": date_posted,
                "Employment Type": job_type,
                "Job ID": job_id,
                "Job Context": job_description,
                "Link": link
            }

            all_job_data.append(job_data)

        except Exception as e:
            print(f"Error processing {link}: {e}")
        finally:
            # To avoid overwhelming the server, wait a bit before the next request
            time.sleep(15)

driver3.quit()

KeyboardInterrupt: 

In [ ]:
# Transform the dict list into a DataFrame
df = pd.DataFrame(all_job_data)

# Save data to CSV file for backup
df.to_excel("Broadcom_job_data.xlsx", index=False)

# Preview the DataFrame
df.head()

In [ ]:
# Note: The promt is in Vietnamese, howeverm the response is in English.
# If you want to change the prompt to English, please change the DEFINE_PROMPT variable below.
DEFINE_PROMPT = """
Từ đoạn text sau, bạn hãy phân tích giúp tôi và trả lời các câu hỏi sau, kết quả trả về cho các variable tương ứng như sau:
- work_category: Danh mục công việc này là gì? (VD: Software Engineer, Data Analyst, ...)
- domain: phân loại mảng làm việc chính (ví dụ: IC Design, Software, Backend, Frontend, Embedded, DevOps, Firmware, System, v.v.) 
- job_description: Mô tả công việc này là cần làm những việc gì?
- job_responsibility: Công việc này cần những trách nhiệm gì?
- job_requirements: Công việc này cần những yêu cầu gì?
- require_tools: Công việc này cần những kỹ năng về công cụ nào?
- require_certificate: Công việc này cần những bằng cấp gì? (Không tính bằng đại học như master, bachelor, ...)
- require_skills: Công việc này cần những kỹ năng gì?
- job_experience: Công việc này cần những kinh nghiệm gì?
- education_level: Bằng cấp học vấn cần thiết cho công việc này là gì? (VD: Bachelor's, Master's, PhD, ...)
- education_major: Chuyên ngành học cần thiết cho công việc này là gì? (VD: Computer Science, Logistic, Engineering..., nếu không được nêu trong mô tả thì dự đoán những chuyên ngành có liên quan nhất tới ngành nghề này giúp mình nha)
----
text: {job_description}
----
Trả về thông tin theo định dạng JSON như sau:
{{
    "job_description": "",
    "job_responsibility": "",
    "job_requirements": "",
    "require_tools": [],
    "require_skills": [],
    "require_certificate": []
    "job_experience": "",
    "education_level": "",
    "education_major": ""
}}
Chỉ trả về thông tin theo dạng trên theo tiếng Anh, không thêm phần giới thiệu hay kết luận nào khác. Ngoài ra đừng trả về dưới dạng markdown.
Ngoài ra, tất đối với require_tools, require_skills, require_certificate, education_level, education_major thì tất cả không viết hoa.
"""


In [ ]:
client = genai.Client(api_key=googleai_key)

In [ ]:
def extract_requirement(job_description):
    chat = client.chats.create(model="gemini-2.0-flash")
    
    prompt = DEFINE_PROMPT.format(job_description=job_description)
    
    response = chat.send_message(
        message=prompt,
    )
    return response.text

In [ ]:
def clean_and_parse_json(raw_response):
    cleaned = raw_response.strip()

    # Remove markdown (```json ... ```)
    cleaned = re.sub(r"^```json\s*|\s*```$", "", cleaned, flags=re.IGNORECASE).strip()

    # Replace None (Python) to null (JSON)
    cleaned = cleaned.replace("None", "null")

    # if "null" return as string then return null as value
    cleaned = re.sub(r'"\s*null\s*"', 'null', cleaned)

    # Sometime it return both version in English and Vietnamese, so we need to remove the Vietnamese version
    if cleaned.count("{") > 1:
        cleaned = cleaned.split("}", 1)[0] + "}"

    # Parse JSON
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("JSONDecodeError:", e)
        print("Error string:", repr(cleaned))
        return {}

In [ ]:
work_category = []
domain = []
job_descriptions = []
job_responsibilities = []
job_requirements = []
tools = []
skills = []
certs = []
job_experience = []
education_level = []
education_major = []

for idx, row in df.iterrows():
    
    try:
        result = extract_requirement(row["Job Context"])
        parsed = clean_and_parse_json(result)

        work_category.append(parsed.get("work_category", ""))
        domain.append(parsed.get("domain", ""))
        job_descriptions.append(parsed.get("job_description", ""))
        job_responsibilities.append(parsed.get("job_responsibility", ""))
        job_requirements.append(parsed.get("job_requirements", ""))
        tools.append(", ".join(x for x in parsed.get("require_tools") or [] if x))
        skills.append(", ".join(x for x in parsed.get("require_skills") or [] if x))
        certs.append(", ".join(x for x in parsed.get("require_certificate") or [] if x))
        job_experience.append(parsed.get("job_experience", ""))
        education_level.append(parsed.get("eduction_level", ""))
        education_major.append(parsed.get("eduction_major", ""))
        

    except Exception as e:
        print(f"Error at row {idx}: {e}")

    # Due to the rpm limitation of Google AI, we need to sleep for 20 seconds after each request
    time.sleep(20)

In [ ]:
df["Work Category"] = work_category
df["Domain"] = domain
df["Job Description New"] = job_descriptions
df["Job Responsibility"] = job_responsibilities
df["Job Requirements"] = job_requirements
df["Job Experience"] = job_experience
df["Education Level"] = education_level
df["Education Major"] = education_major
df["Require Tools"] = tools
df["Require Skills"] = skills
df["Require Certificate"] = certs

In [ ]:
# Save data to CSV file for backup
df.to_csv("Broadcom_job_data_v2.csv", index=False, encoding='utf-8-sig')

In [4]:
wb = load_workbook("Broadcom_job_data_v2.xlsx")
ws = wb.active
today = time.strftime("%Y-%m-%d")
company = "Broadcom"

# Định nghĩa độ rộng cột
column_settings = {
    ("Job Title", "Job Description", "Job Description New", "Job Responsibility", "Job Requirements", "Job Context"): 50,
    ("Company", "Job Location", "Date Posted", "Employment Type", "Job ID"): 30,
    ("Work Category", "Domain"): 20,
    ("Require Tools", "Require Skills", "Require Certificate"): 25,
    ("Job Experience", "Education Level", "Education Major"): 15,
    ("Link",): 50,
}

# Thiết lập độ rộng cột và bật wrap_text
for cell in ws[1]:  # Lặp qua hàng đầu tiên (header)
    if cell.value:
        header = cell.value.strip()
        for headers, width in column_settings.items():
            if header in headers:
                col_letter = get_column_letter(cell.column)
                ws.column_dimensions[col_letter].width = width
                print(f"Set width of column {col_letter} (header '{header}') to {width}")

                # Bật wrap_text cho tất cả các ô trong cột
                for row in ws.iter_rows(min_col=cell.column, max_col=cell.column, min_row=2, max_row=ws.max_row):
                    for c in row:
                        c.alignment = Alignment(wrap_text=True)

# Căn chỉnh toàn bộ sheet
for row in ws.iter_rows():
    for cell in row:
        if cell.alignment:
            cell.alignment = Alignment(
                horizontal=cell.alignment.horizontal or "left",
                vertical="top",
                text_rotation=cell.alignment.text_rotation,
                wrap_text=cell.alignment.wrap_text,
                shrink_to_fit=cell.alignment.shrink_to_fit,
                indent=cell.alignment.indent
            )

# Hàm tự động điều chỉnh chiều cao hàng dựa trên wrap_text
def auto_adjust_row_heights(worksheet, base_height=15, words_per_line=7):
    for row in worksheet.iter_rows(min_row=1, max_row=worksheet.max_row):
        max_lines = 1
        for cell in row:
            if cell.alignment and cell.alignment.wrap_text and cell.value:
                lines = str(cell.value).split('\n')
                estimated_lines = sum((len(line.split()) + words_per_line - 1) // words_per_line for line in lines)
                max_lines = max(max_lines, estimated_lines)
        worksheet.row_dimensions[row[0].row].height = base_height * max_lines

# Điều chỉnh chiều cao hàng
auto_adjust_row_heights(ws)

# Lưu file Excel mới
output_file = f"./{company}_job_posts_allcountries_raw_{today}.xlsx"
wb.save(output_file)

Set width of column A (header 'Company') to 30
Set width of column B (header 'Job Title') to 50
Set width of column C (header 'Job Location') to 30
Set width of column D (header 'Date Posted') to 30
Set width of column E (header 'Employment Type') to 30
Set width of column F (header 'Job ID') to 30
Set width of column G (header 'Job Description') to 50
Set width of column H (header 'Job Context') to 50
Set width of column I (header 'Job Description New') to 50
Set width of column J (header 'Job Responsibility') to 50
Set width of column K (header 'Job Requirements') to 50
Set width of column L (header 'Job Experience') to 15
Set width of column M (header 'Education Level') to 15
Set width of column N (header 'Education Major') to 15
Set width of column O (header 'Require Tools') to 25
Set width of column P (header 'Require Skills') to 25
Set width of column Q (header 'Require Certificate') to 25
Set width of column R (header 'Work Category') to 20
Set width of column S (header 'Domain'